In [1]:
import os
import pickle
import re
import sys
import numpy as np
sys.path.append('../../src/')

from utils import loader

from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm
from pred_evaluation import get_all_and_matched_uuid_lsts

from mask_dataset import MaskedDataset
dataset = MaskedDataset(model_name="mbert", reload=False)
root = "/home/xzhao/workspace/probing-mulitlingual/datasets/2018_dump_wiki_cache/sub_obj_ggrep"
dump_root = "/disk/xzhao/datasets/wikipedia_2018_octnov/sub_obj_ggrep_processed_result_dump"


Load pre-saved file /home/xzhao/workspace/probing-mulitlingual/result/dataset-mbert/mlama-dataset.pkl for MaskedDataset instance


In [2]:
lang2rel2all_uuid, lang2rel2matched_uuid = get_all_and_matched_uuid_lsts(dataset=dataset, reload=False)
lang2tokenized2sub_uri = defaultdict(lambda: defaultdict(set))
for sub_uri, lang2info in dataset.get_sub_info().items():
    for lang, info in lang2info.items():
        lang2tokenized2sub_uri[lang][' '.join(info['sub_tokens'])].add(sub_uri)
lang2tokenized2obj_uri = defaultdict(lambda: defaultdict(set))
for obj_uri, lang2info in dataset.get_obj_info().items():
    for lang, info in lang2info.items():
        lang2tokenized2obj_uri[lang][' '.join(info['obj_tokens'])].add(obj_uri)

In [12]:
lang = 'zh'
tokenized2sub_uri = lang2tokenized2sub_uri[lang]
tokenized2obj_uri = lang2tokenized2obj_uri[lang]
rel2all_uuid = lang2rel2all_uuid[lang]
rel2matched_uuid = lang2rel2matched_uuid[lang]

lang2rel2all_uuid, lang2rel2matched_uuid = get_all_and_matched_uuid_lsts(dataset=dataset, reload=False)
rel2matched_uuid = lang2rel2matched_uuid[lang]
rel2all_uuid = lang2rel2all_uuid[lang]

In [26]:
sub_info = dataset.get_sub_info()
for sub_uri in sub_info:
    if 'zh' in sub_info[sub_uri] and '石勒' in sub_info[sub_uri]['zh']['sub']:
        print(sub_uri, sub_info[sub_uri]['zh'])
obj_info = dataset.get_obj_info()
for obj_uri in obj_info:
    if 'zh' in obj_info[obj_uri] and obj_info[obj_uri]['zh']['obj'] == "汉堡":
        print(obj_uri, obj_info[obj_uri]['zh'])

Q1194 {'sub': '石勒苏益格-荷尔斯泰因', 'sub_ids': [5824, 2620, 6702, 5736, 4580, 118, 6761, 3461, 4332, 4952, 3000], 'sub_tokens': ['石', '勒', '苏', '益', '格', '-', '荷', '尔', '斯', '泰', '因']}
Q1055 {'obj': '汉堡', 'obj_ids': [4877, 3110], 'obj_tokens': ['汉', '堡']}


In [23]:
uuid_infos = dataset.get_uuid_info_per_lang()['zh']
for uuid_info in uuid_infos:
    if uuid_info['sub_uri'] == "Q1194":
        print(uuid_info)

{'obj_uri': 'Q1055', 'sub_uri': 'Q1194', 'rel_uri': 'P47', 'obj': '汉堡', 'sub': '石勒苏益格-荷尔斯泰因', 'rel': '[X]与[Y]共享边界。', 'uuid': '0c7fdafa-c75b-4bb8-af41-eb0313bf4a55'}
{'obj_uri': 'Q1707', 'sub_uri': 'Q1194', 'rel_uri': 'P36', 'obj': '基尔', 'sub': '石勒苏益格-荷尔斯泰因', 'rel': '[X]的大写字母为[Y]。', 'uuid': '81edd07f-1ed8-4b3e-94a2-beb7ff999337'}


In [4]:
tokenized2sub_uri['054 型 护 卫 舰']

['0d31b202-a4dc-42e0-8067-13938039b7f3',
 'd167615d-d364-4f56-b2a1-e56e8d4e5a81',
 'f54e537e-7bbd-4868-9f56-7e5d53795dd1',
 '33155b9b-2904-4154-ab78-860da2b10690',
 'f1e69016-2e90-432f-b36d-f6dc0f3897db',
 'f66daeb4-9003-4a2b-a0be-cc13ea1b82c1',
 '555ae6eb-93f3-4ecc-9bc2-fe4cc848864e']

In [27]:
lang_root = os.path.join(root, 'zh')
text2id = {}
suburi2lineid = defaultdict(set)
objuri2lineid = defaultdict(set)
for subdir in tqdm(os.listdir(lang_root)):
    subdir = os.path.join(lang_root, subdir)
    for idx in range(10):
        obj_match_fn = os.path.join(subdir, f"object_match.txt.0{idx}")
        sub_match_fn = os.path.join(subdir, f"subject_match.txt.0{idx}")            
        file_id = subdir.split("/")[-1] + f":{idx}"
        with open(sub_match_fn, 'r') as fp:
            for line in fp:
                line_id, *words = re.match(r'^(\d+):(.*)$', line.strip()).groups()
                for word in map(str.strip, words[0].split('\t')):
                    for sub_uri in tokenized2sub_uri[word]:
                        unique_text = f"{file_id}:{line_id}"
                        if unique_text not in text2id:
                            text2id.update({unique_text: len(text2id)})
                        suburi2lineid[sub_uri].add(text2id[unique_text])
        with open(obj_match_fn, 'r') as fp:
            for line in fp:
                line_id, *words = re.match(r'^(\d+):(.*)$', line.strip()).groups()
                for word in map(str.strip, words[0].split('\t')):
                    for obj_uri in tokenized2obj_uri[word]:
                        unique_text = f"{file_id}:{line_id}"
                        if unique_text not in text2id:
                            text2id.update({unique_text: len(text2id)})
                        objuri2lineid[obj_uri].add(text2id[unique_text])

100%|██████████| 9/9 [00:37<00:00,  4.14s/it]


In [28]:
objuri2lineid['Q1055']

{'zhwiki-20181101-pages-articles5.xml-p2771087p4271087:5:12857',
 'zhwiki-20181101-pages-articles6.xml-p4731444p6231444:3:34503',
 'zhwiki-20181101-pages-articles5.xml-p2771087p4271087:7:50158',
 'zhwiki-20181101-pages-articles5.xml-p2771087p4271087:9:145106',
 'zhwiki-20181101-pages-articles5.xml-p2771087p4271087:3:1585',
 'zhwiki-20181101-pages-articles6.xml-p4731444p6231444:0:15218',
 'zhwiki-20181101-pages-articles4.xml-p1154618p2654618:1:977',
 'zhwiki-20181101-pages-articles3.xml-p544653p1154617:0:48913',
 'zhwiki-20181101-pages-articles6.xml-p4731444p6231444:6:63444',
 'zhwiki-20181101-pages-articles5.xml-p2771087p4271087:5:81858',
 'zhwiki-20181101-pages-articles5.xml-p2771087p4271087:1:103663',
 'zhwiki-20181101-pages-articles5.xml-p4271087p4731436:0:6680',
 'zhwiki-20181101-pages-articles2.xml-p162888p544640:2:15912',
 'zhwiki-20181101-pages-articles2.xml-p162888p544640:9:25375',
 'zhwiki-20181101-pages-articles6.xml-p4731444p6231444:4:63251',
 'zhwiki-20181101-pages-articles

In [ ]:
for rel in rel2matched_uuid:
    for uuid in rel2matched_uuid[rel]:
        uuid_info = dataset.uuid_info_plain[uuid]
        sub_uri = uuid_info['sub_uri']
        obj_uri = uuid_info['obj_uri']
        intersection = suburi2lineid[sub_uri].intersection(objuri2lineid[obj_uri])
        print(suburi2lineid[sub_uri].intersection(objuri2lineid[obj_uri]))
            